In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce
import tabula
%matplotlib inline

* participants in Wave IV are 24 - 32 years old
* find the percentage of the population that achieved each level of academic attainment after hs 
* Census data in thousands

Census

In [ ]:
census_all = pd.read_excel(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\Census Educational Attainment - 2008.xls", 'all')
census_all.head()

In [ ]:
censusall_sub = census_all.iloc[3:5]
censusall_sub

In [ ]:
censusall_sub.loc['Total'] = censusall_sub.iloc[:, :].sum()

In [ ]:
censusall_sub.reset_index(drop = False)

In [ ]:
census_pct = censusall_sub.iloc[1:, 1:].div(censusall_sub['Total'], axis = 0)

In [ ]:
for col in censusall_sub.columns[2:] :   
    census_pct[col] = (censusall_sub[col]/censusall_sub['Total'])

In [ ]:
census_pct = round(census_pct, 4)
census_pct

Merge Wave AIDs

In [ ]:
wave1 = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0001\21600-0001-Data.tsv", sep = '\t', usecols = ['AID'])
wave3educ = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0016\21600-0016-Data.tsv", sep = '\t', usecols = ['AID'])
wave3grad = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0017\21600-0017-Data.tsv", sep = '\t', usecols = ['AID'])
wave4 = pd.read_csv(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0022\21600-0022-Data.tsv", sep = '\t', usecols = ['AID'])

In [ ]:
print('Count of Responses per Wave')
print('Wave 1: ', len(wave1))
print('Wave 3 education: ', len(wave3educ))
print('Wave 3 graduation: ', len(wave3grad))
print('Wave 4: ', len(wave4))

In [ ]:
waves = [wave1, wave3educ, wave3grad, wave4]
AID = reduce(lambda left, right: pd.merge(left, right, on = 'AID', how = 'inner'), waves)
len(AID)

Highest Education Level

In [ ]:
highest_educ = pd.read_excel(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\Wave 4 Questionaire .xlsx", 'highest_educ_level')
highest_educ = highest_educ.rename(columns = highest_educ.iloc[0])
highest_educ = highest_educ.drop(highest_educ.index[0])
highest_educ = highest_educ.drop(['Value'], axis = 1)
highest_educ

In [ ]:
census_pct

In [ ]:
most_recent_degree = pd.read_excel(r"C:\Users\mnorm\Documents\NSS\NSS-Capstone\data\Wave 4 Questionaire .xlsx", 'most_recent_degree')
most_recent_degree = most_recent_degree.rename(columns = most_recent_degree.iloc[0])
most_recent_degree = most_recent_degree.drop(most_recent_degree.index[0])
most_recent_degree = most_recent_degree.drop(['Value'], axis =1)
most_recent_degree
#legitimate skip represents those who did not complete high school

In [ ]:
educ = pd.read_csv(r"\Users\mnorm\Documents\NSS\NSS-Capstone\data\National Longitudinal Study of Adolescent to Adult Health\ICPSR_21600\DS0022\21600-0022-Data.tsv", sep = '\t', usecols = ['AID', 'H4ED2'])
educ = educ.rename(columns = {'AID':'AID', 'H4ED2': 'highest_educ_level'})
educ